In [1]:
import gc
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE_H':256, # slow : 208, 256
    'IMG_SIZE_W':512,
    'EPOCHS':100,
    'LEARNING_RATE':1e-6,
    'BATCH_SIZE':4,
    'SEED':909
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Focal Loss

In [5]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction=self.reduction)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

## Custom Dataset

In [6]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)        
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            img = cv2.resize(img, (CFG['IMG_SIZE_W'], CFG['IMG_SIZE_H']))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

## Modeling

In [7]:
from torchvision.models.video import r3d_18, mc3_18, r2plus1d_18
from torchvision.models.video import mvit_v1_b, mvit_v2_s

In [8]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=None):
        super(BaseModel, self).__init__()
        self.base_model = r3d_18(weights='KINETICS400_V1')
        # self.base_model = mc3_18(weights='KINETICS400_V1')
        # self.base_model = r2plus1d_18(weights='KINETICS400_V1')
        
        self.classifier =  nn.Sequential(
            nn.Linear(400, 256),
            nn.ReLU(),
            nn.Dropout(p=0.35),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):        
        x = self.base_model(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [10]:
class BaseModel2(nn.Module):
    def __init__(self, num_classes=None):
        super(BaseModel2, self).__init__()
        # self.base_model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True).to(device) #2048
        self.base_model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_s', pretrained=True).to(device) #2048

        self.base_model.blocks[5].proj = nn.Sequential(
            nn.Linear(2048, 400),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(400, num_classes)
        ).to(device)
        
    def forward(self, x):        
        x = self.base_model(x)        
        return x

In [11]:
class BaseModel3(nn.Module):
    def __init__(self, num_classes=None):
        super(BaseModel3, self).__init__()
        # self.base_model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True).to(device) #2048
        self.base_model = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_m', pretrained=True).to(device) #2048

        self.base_model.blocks[5].proj = nn.Sequential(
            nn.Linear(2048, 400),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(400, num_classes)
        ).to(device)
        
    def forward(self, x):        
        x = self.base_model(x)        
        return x

## Trainer

In [12]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, weights):
    model.to(device)
    # criterion = nn.CrossEntropyLoss(weight=weights).to(device)
    criterion = FocalLoss(weight=weights).to(device)
    
    best_val_score = 0
    best_val_loss = 9999999
    best_model = None
    
    # Early Stop
    patience_limit = 4
    patience = 0

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            # scheduler.step(_val_score)
            scheduler.step(_val_loss)
            
        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_val_score = _val_score
            best_model = model
            patience = 0
        else:
            patience += 1
            if patience >= patience_limit:
                break
                
    print(f'Best Val Loss : [{best_val_loss:.5f}] Best Val F1 : [{best_val_score:.5f}]')
    return best_model, best_val_score, best_val_loss

In [13]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='weighted')
    return _val_loss, _val_score

## Load Data

In [14]:
df = pd.read_csv('./datasets/train_CE-W-T_aug.csv')

error_data = [8, 124, 330, 387, 486, 1113, 1533, 2292,
                8+2698, 124+2698, 330+2698, 387+2698, 486+2698, 1113+2698, 1533+2698, 2292+2698]

for error in error_data:
    df = df.drop(error)
    
df = df.reset_index(drop=True)

## Under Sampling

In [13]:
from sklearn.utils import resample

In [ ]:
df_crash_ego = df.loc[:, ['sample_id', 'video_path', 'crash_ego']]
print(df_crash_ego["crash_ego"].value_counts(dropna=False))

df_crash_ego_0 = df_crash_ego[df_crash_ego['crash_ego'] == 0]
df_crash_ego_1 = df_crash_ego[df_crash_ego['crash_ego'] == 1]
df_crash_ego_2 = df_crash_ego[df_crash_ego['crash_ego'] == 2]

n_0 = len(df_crash_ego_0)
n_1 = len(df_crash_ego_1)
n_2 = len(df_crash_ego_2)

df_class_0_downsampled = resample(df_crash_ego_0, 
                                 replace=False,
                                 n_samples=n_1,     
                                 random_state=CFG['SEED'])

df_crash_ego = pd.concat([df_class_0_downsampled, df_crash_ego_1, df_crash_ego_2])
print(df_crash_ego["crash_ego"].value_counts(dropna=False))

In [ ]:
df_weather = df.loc[:, ['sample_id', 'video_path', 'weather']]
df_weather = df_weather[df_weather['weather']>-1]
print(df_weather["weather"].value_counts(dropna=False))

df_weather_0 = df_weather[df_weather['weather'] == 0]
df_weather_1 = df_weather[df_weather['weather'] == 1]
df_weather_2 = df_weather[df_weather['weather'] == 2]

n_0 = len(df_weather_0)
n_1 = len(df_weather_1)
n_2 = len(df_weather_2)

df_class_0_downsampled = resample(df_weather_0, 
                                 replace=False,
                                 n_samples=n_1,     
                                 random_state=CFG['SEED'])

df_weather = pd.concat([df_class_0_downsampled, df_weather_1, df_weather_2])
print(df_weather["weather"].value_counts(dropna=False))

In [10]:
df_timing = df.loc[:, ['sample_id', 'video_path', 'timing']]
df_timing = df_timing[df_timing['timing']>-1]
print(df_timing["timing"].value_counts(dropna=False))

df_timing_0 = df_timing[df_timing['timing'] == 0]
df_timing_1 = df_timing[df_timing['timing'] == 1]

n_0 = len(df_timing_0)
n_1 = len(df_timing_1)

df_class_0_downsampled = resample(df_timing_0, 
                                 replace=False,
                                 n_samples=n_1,     
                                 random_state=CFG['SEED'])

df_timing = pd.concat([df_class_0_downsampled, df_timing_1])
print(df_timing["timing"].value_counts(dropna=False))

NameError: name 'df' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 303


In [15]:
df_crash_ego = df.loc[:, ['sample_id', 'video_path', 'crash_ego']]

df_weather = df.loc[:, ['sample_id', 'video_path', 'weather']]
df_weather = df_weather[df_weather['weather']>-1]

df_timing = df.loc[:, ['sample_id', 'video_path', 'timing']]
df_timing = df_timing[df_timing['timing']>-1]

print(len(df_crash_ego), len(df_weather), len(df_timing))

5380 1828 1828


## Class Weights

In [16]:
from sklearn.utils.class_weight import compute_class_weight

crash_ego_labels = [i for i in df_crash_ego['crash_ego']]
crash_ego_labels.sort()
crash_ego_class_weights = compute_class_weight(class_weight = "balanced", classes=np.unique(crash_ego_labels), y=crash_ego_labels)
crash_ego_class_weights = torch.FloatTensor(crash_ego_class_weights).to(device)

weather_labels = [i for i in df_weather['weather']]
weather_labels.sort()
weather_class_weights = compute_class_weight(class_weight = "balanced", classes=np.unique(weather_labels), y=weather_labels)
weather_class_weights = torch.FloatTensor(weather_class_weights).to(device)

timing_labels = [i for i in df_timing['timing']]
timing_labels.sort()
timing_class_weights = compute_class_weight(class_weight = "balanced", classes=np.unique(timing_labels), y=timing_labels)
timing_class_weights = torch.FloatTensor(timing_class_weights).to(device)

crash_ego_class_weights, weather_class_weights, timing_class_weights

(tensor([0.5049, 1.8262, 2.1198], device='cuda:0'),
 tensor([0.4261, 2.3618, 4.3524], device='cuda:0'),
 tensor([0.5656, 4.3113], device='cuda:0'))

In [17]:
train_crash_ego, val_crash_ego, _, _ = train_test_split(df_crash_ego, df_crash_ego['crash_ego'], test_size=0.2, random_state=CFG['SEED'], stratify=df_crash_ego['crash_ego'])
print(len(train_crash_ego), len(val_crash_ego))

train_weather, val_weather, _, _ = train_test_split(df_weather, df_weather['weather'], test_size=0.2, random_state=CFG['SEED'], stratify=df_weather['weather'])
print(len(train_weather), len(val_weather))

train_timing, val_timing, _, _ = train_test_split(df_timing, df_timing['timing'], test_size=0.2, random_state=CFG['SEED'], stratify=df_timing['timing'])
print(len(train_timing), len(val_timing))

4304 1076
1462 366
1462 366


## Train

In [17]:
# Crash & Ego 분류
train_dataset = CustomDataset(train_crash_ego['video_path'].values, train_crash_ego['crash_ego'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_crash_ego['video_path'].values, val_crash_ego['crash_ego'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = BaseModel2(num_classes=3)
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params = model.parameters(), lr=2e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-12, verbose=True)
crash_ego_model, crash_ego_best_score, crash_ego_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, crash_ego_class_weights)

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리

torch.save(crash_ego_model.module.state_dict(), f'./models/CE-W-T_[x3d_m]_[crash_ego]_[score{crash_ego_best_score:.4f}]_[loss{crash_ego_best_loss:.4f}].pt')

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main



Epoch [1], Train Loss : [0.46031] Val Loss : [0.37995] Val F1 : [0.69023]



Epoch [2], Train Loss : [0.32626] Val Loss : [0.18579] Val F1 : [0.82922]



Epoch [3], Train Loss : [0.21668] Val Loss : [0.11310] Val F1 : [0.88218]



Epoch [4], Train Loss : [0.16701] Val Loss : [0.08417] Val F1 : [0.90409]


...


Epoch [32], Train Loss : [0.01083] Val Loss : [0.02193] Val F1 : [0.96204]



Epoch [33], Train Loss : [0.01302] Val Loss : [0.02416] Val F1 : [0.96262]



Epoch [34], Train Loss : [0.01108] Val Loss : [0.02266] Val F1 : [0.95985]



Epoch [35], Train Loss : [0.01038] Val Loss : [0.02384] Val F1 : [0.96840]
Epoch 00035: reducing learning rate of group 0 to 2.5000e-07.



Epoch [36], Train Loss : [0.01012] Val Loss : [0.02545] Val F1 : [0.96145]
Best Val Loss : [0.02193] Best Val F1 : [0.96204]


In [16]:
# Weather 분류
train_dataset = CustomDataset(train_weather['video_path'].values, train_weather['weather'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_weather['video_path'].values, val_weather['weather'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = BaseModel2(num_classes=3)
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-12, verbose=True)
weather_model, weather_best_score, weather_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, weather_class_weights)

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리  

torch.save(weather_model.module.state_dict(), f'./models/CE-W-T_[x3d_m]_[weather]_[score{weather_best_score:.4f}]_[loss{weather_best_loss:.4f}].pt')

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main



Epoch [1], Train Loss : [0.29210] Val Loss : [0.10362] Val F1 : [0.85936]



Epoch [2], Train Loss : [0.08388] Val Loss : [0.07803] Val F1 : [0.87541]



Epoch [3], Train Loss : [0.04715] Val Loss : [0.16004] Val F1 : [0.70670]



Epoch [4], Train Loss : [0.02460] Val Loss : [0.07970] Val F1 : [0.85905]


...


Epoch [6], Train Loss : [0.01003] Val Loss : [0.02386] Val F1 : [0.94267]



Epoch [7], Train Loss : [0.00528] Val Loss : [0.04048] Val F1 : [0.92977]



Epoch [8], Train Loss : [0.00385] Val Loss : [0.08026] Val F1 : [0.94637]



Epoch [9], Train Loss : [0.00151] Val Loss : [0.03229] Val F1 : [0.92996]
Epoch 00009: reducing learning rate of group 0 to 2.5000e-05.



Epoch [10], Train Loss : [0.00127] Val Loss : [0.02389] Val F1 : [0.95791]
Best Val Loss : [0.02386] Best Val F1 : [0.94267]


In [31]:
# Timing 분류
train_dataset = CustomDataset(train_timing['video_path'].values, train_timing['timing'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_timing['video_path'].values, val_timing['timing'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = BaseModel2(num_classes=2)
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-12, verbose=True)
timing_model, timing_best_score, timing_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, timing_class_weights)

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리  

torch.save(timing_model.module.state_dict(), f'./models/CE-W-T_[x3d_m]_[timing]_[score{timing_best_score:.4f}]_[loss{timing_best_loss:.4f}].pt')

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


KeyboardInterrupt: 

## Inference

In [18]:
test = pd.read_csv('./datasets/test.csv')
test['video_path'] = test['video_path'].apply(lambda x: x.replace('./test/', './datasets/test/'))

In [19]:
# test_dataset = CustomDataset(test['video_path'].values, None, test_transforms)
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

In [20]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [22]:
crash_ego_model_weights = torch.load('./models/x3d_s-0.6735-208_416/CE-W-T_[x3d_s]_[crash_ego]_[score0.9685]_[loss0.0138].pt')
crash_ego_model = BaseModel2(num_classes=3)
crash_ego_model.load_state_dict(crash_ego_model_weights)

weather_model_weights = torch.load('./models/CE-W-T_[x3d_m]_[weather]_[score0.9427]_[loss0.0239].pt')
weather_model = BaseModel3(num_classes=3)
weather_model.load_state_dict(weather_model_weights)

timing_model_weights = torch.load('./models/x3d_s-0.67804-208_416-pesudo0.9/CE-W-T_[x3d_s]_[timing]_[score0.9820]_[loss0.0053].pt')
timing_model = BaseModel2(num_classes=2)
timing_model.load_state_dict(timing_model_weights)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


<All keys matched successfully>

In [23]:
crash_ego_preds = inference(crash_ego_model, test_loader, device)

In [24]:
weather_preds = inference(weather_model, test_loader, device)

In [25]:
timing_preds = inference(timing_model, test_loader, device)

In [26]:
label_dict = {1:[1,0,0], 2:[1,0,1], 3:[1,1,0], 4:[1,1,1], 5:[1,2,0],6:[1,2,1],7:[2,0,0],8:[2,0,1],9:[2,1,0],10:[2,1,1],11:[2,2,0],12:[2,2,1]}
label_dict_rev = {(1,0,0):1, (1,0,1):2, (1,1,0):3, (1,1,1):4, (1,2,0):5, (1,2,1):6, (2,0,0):7, (2,0,1):8, (2,1,0):9, (2,1,1):10, (2,2,0):11, (2,2,1):12}

In [27]:
preds = []
for crash_ego_pred, weather_pred, timing_pred in zip(crash_ego_preds, weather_preds, timing_preds):
    if crash_ego_pred == 0: # 충돌 X
        preds.append(0)
    else:
        temp = (crash_ego_pred, weather_pred, timing_pred)
        preds.append(label_dict_rev[temp])

In [28]:
submit = pd.read_csv('./datasets/sample_submission.csv')

In [29]:
submit['label'] = preds
submit.head()

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [30]:
# submit.to_csv(f'./submits/CE-W-T_{crash_ego_best_score:.4f}_{weather_best_score:.4f}_{timing_best_score:.4f}.csv', index=False)
submit.to_csv(f'./submits/CE-W-T_0.9685_0.9427_0.9820.csv', index=False)